<a href="https://colab.research.google.com/github/ash827/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/AG_LS_DS_432_Convolution_Neural_Networks_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

- <a href="#p1">Part 1:</a> Pre-Trained Model
- <a href="#p2">Part 2:</a> Custom CNN Model
- <a href="#p3">Part 3:</a> CNN with Data Augmentation


You will apply three different CNN models to a binary image classification model using Keras. Classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

|Mountain (+)|Forest (-)|
|---|---|
|![](https://github.com/ash827/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module2-convolutional-neural-networks/data/mountain/art1131.jpg?raw=1)|![](https://github.com/ash827/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module2-convolutional-neural-networks/data/forest/cdmc317.jpg?raw=1)|

The problem is realively difficult given that the sample is tiny: there are about 350 observations per class. This sample size might be something that you can expect with prototyping an image classification problem/solution at work. Get accustomed to evaluating several differnet possible models.

# Pre - Trained Model
<a id="p1"></a>

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D()
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

```

The `include_top` parameter in `ResNet50` will remove the full connected layers from the ResNet model. The next step is to turn off the training of the ResNet layers. We want to use the learned parameters without updating them in future training passes. 

```python
for layer in resnet.layers:
    layer.trainable = False
```

Using the Keras functional API, we will need to additional additional full connected layers to our model. We we removed the top layers, we removed all preivous fully connected layers. In other words, we kept only the feature processing portions of our network. You can expert with additional layers beyond what's listed here. The `GlobalAveragePooling2D` layer functions as a really fancy flatten function by taking the average of each of the last convolutional layer outputs (which is two dimensional still). 

```python
x = res.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(res.input, predictions)
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pretrained layers from resnet
4. Report your model's accuracy

## Load in Data

![skimage-logo](https://scikit-image.org/_static/img/logo.png)

Check out out [`skimage`](https://scikit-image.org/) for useful functions related to processing the images. In particular checkout the documentation for `skimage.io.imread_collection` and `skimage.transform.resize`.

In [0]:
# Import libraries 

import os
import sys
import cv2
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt 

from tensorflow import keras
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
 
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model # This is the functional API
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img 
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils.np_utils import to_categorical 

from sklearn import metrics
from sklearn.metrics import confusion_matrix
 
resnet = ResNet50(weights='imagenet', include_top=False)

In [0]:
# Trainable set to False

for layer in resnet.layers:
    layer.trainable = False

In [0]:
x = resnet.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x) # 1 for binary classification (otherwise no. of classes). for binary classification, use sigmoid, else softmax
model = Model(resnet.input, predictions)

In [0]:
base_learning_rate = 0.001
model.compile(optimizer = tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss = tf.keras.losses.BinaryCrossentropy(from_logits = True),
              metrics = ['accuracy'])

In [19]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [21]:
# Using git clone to add in my repo to load in the data from the Data folder

!git clone https://github.com/ash827/DS-Unit-4-Sprint-3-Deep-Learning.git

fatal: destination path 'DS-Unit-4-Sprint-3-Deep-Learning' already exists and is not an empty directory.


In [0]:
# x and y labels for training data

FILEPATH = 'DS-Unit-4-Sprint-3-Deep-Learning/module2-convolutional-neural-networks/data/train/'
CATEGORIES = ['forest', 'mountain']
training_data = []
IMG_SIZE = 224

def create_training_data():
  for category in CATEGORIES:
    path = os.path.join(FILEPATH, category) 
    class_num = CATEGORIES.index(category)
    print(path, class_num)
    for img in os.listdir(path):
      print(img)
      try:
        #if not img.startswith(\".jpg\"):
        img_array = cv2.imread(os.path.join(path, img))
        new_img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        training_data.append([new_img_array, class_num])
      except Exception as e:
        pass
  X = []
  y = []

  for features, label in training_data:
    X.append(features)
    y.append(label)

    X = np.array(X)
    y = to_categorical(np.array(y))
    print('------- Shape of X, y data -------')
    print(X.shape, y.shape)
    return X, y

In [23]:
X_train, y_train = create_training_data()

DS-Unit-4-Sprint-3-Deep-Learning/module2-convolutional-neural-networks/data/train/forest 0
land863.jpg
for148.jpg
natu860.jpg
nat1033.jpg
natu308.jpg
for110.jpg
land849.jpg
natu172.jpg
nat263.jpg
natu170.jpg
natu28.jpg
for149.jpg
urb767.jpg
for114.jpg
nat1128.jpg
natu634.jpg
nat719.jpg
natu29.jpg
natu2.jpg
text44.jpg
text48.jpg
for153.jpg
land810.jpg
land215.jpg
nat1027.jpg
natu325.jpg
natu396.jpg
nat1095.jpg
natu871.jpg
for25.jpg
natu322.jpg
land726.jpg
for87.jpg
land871.jpg
for65.jpg
for84.jpg
nat239.jpg
text47.jpg
land217.jpg
natu913.jpg
land922.jpg
text49.jpg
for96.jpg
for67.jpg
land864.jpg
text103.jpg
text50.jpg
nat835.jpg
text29.jpg
natu72.jpg
natu723.jpg
text119.jpg
nat286.jpg
for79.jpg
natu423.jpg
for60.jpg
land861.jpg
nat373.jpg
natu26.jpg
land809.jpg
nat349.jpg
text43.jpg
natu949.jpg
text63.jpg
for136.jpg
natu847.jpg
natu863.jpg
text42.jpg
for143.jpg
for47.jpg
natu158.jpg
natu707.jpg
nat452.jpg
for85.jpg
land867.jpg
natu15.jpg
for50.jpg
nat223.jpg
text13.jpg
nat324.jpg
nat204

In [25]:
FILEPATH = 'DS-Unit-4-Sprint-3-Deep-Learning/module2-convolutional-neural-networks/data/validation/'
CATEGORIES = ['forest', 'mountain']
validation_data = []
IMG_SIZE = 224

def create_validation_data():
  for category in CATEGORIES:
    path = os.path.join(FILEPATH, category) 
    class_num = CATEGORIES.index(category)
    print(path, class_num)
    for img in os.listdir(path):
      print(img)
      try:
        #if not img.startswith(\".jpg\"):
        img_array = cv2.imread(os.path.join(path, img))
        new_img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        training_data.append([new_img_array, class_num])
      except Exception as e:
        pass
  X = []
  y = []

  for features, label in training_data:
    X.append(features)
    y.append(label)

    X = np.array(X)
    y = to_categorical(np.array(y))
    print('------- Shape of X, y data -------')
    print(X.shape, y.shape)
    return X, y

X_val, y_val = create_validation_data()

DS-Unit-4-Sprint-3-Deep-Learning/module2-convolutional-neural-networks/data/validation/forest 0
cdmc291.jpg
cdmc547.jpg
cdmc317.jpg
bost190.jpg
cdmc338.jpg
cdmc12.jpg
cdmc351.jpg
cdmc313.jpg
cdmc388.jpg
cdmc278.jpg
cdmc284.jpg
cdmc533.jpg
cdmc363.jpg
cdmc271.jpg
cdmc333.jpg
bost100.jpg
natc4.jpg
cdmc348.jpg
cdmc277.jpg
cdmc358.jpg
bost98.jpg
cdmc507.jpg
natc52.jpg
cdmc430.jpg
cdmc315.jpg
cdmc368.jpg
cdmc344.jpg
bost102.jpg
natc49.jpg
cdmc451.jpg
cdmc293.jpg
natc37.jpg
bost103.jpg
.DS_Store
cdmc281.jpg
natc12.jpg
cdmc414.jpg
n18075.jpg
cdmc290.jpg
natc13.jpg
cdmc101.jpg
cdmc415.jpg
cdmc352.jpg
cdmc283.jpg
cdmc280.jpg
cdmc494.jpg
cdmc556.jpg
cdmc375.jpg
cdmc458.jpg
cdmc282.jpg
cdmc562.jpg
cdmc306.jpg
cdmc319.jpg
cdmc359.jpg
nat1266.jpg
cdmc331.jpg
cdmc385.jpg
bost101.jpg
cdmc292.jpg
cdmc377.jpg
cdmc318.jpg
DS-Unit-4-Sprint-3-Deep-Learning/module2-convolutional-neural-networks/data/validation/mountain 1
n344032.jpg
n371076.jpg
n371077.jpg
n347077.jpg
n213056.jpg
n213096.jpg
cdmc181.jpg
n1

In [36]:
# defining the epochs and batch size fitting the model
epochs = 10

batch_size=10

stop = EarlyStopping(monitor='val_loss', mode='min', min_delta=0.001, patience=3, verbose=1)
filepath = 'cnn_practice'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, mode='auto', save_freq='epoch')

history = model.fit(
          X_train, y_train,
          epochs = epochs,
          validation_data = (X_val, y_val),
          verbose = 1,
          callbacks = [stop, checkpoint],
          batch_size = batch_size)

Epoch 1/10
1/1 [==============================] - ETA: 0s - loss: 0.3133 - accuracy: 1.0000
Epoch 00001: saving model to cnn_practice
INFO:tensorflow:Assets written to: cnn_practice/assets
1/1 [==============================] - 14s 14s/step - loss: 0.3133 - accuracy: 1.0000 - val_loss: 0.3133 - val_accuracy: 1.0000
Epoch 2/10
1/1 [==============================] - ETA: 0s - loss: 0.3133 - accuracy: 1.0000
Epoch 00002: saving model to cnn_practice
INFO:tensorflow:Assets written to: cnn_practice/assets
1/1 [==============================] - 14s 14s/step - loss: 0.3133 - accuracy: 1.0000 - val_loss: 0.3133 - val_accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - ETA: 0s - loss: 0.3133 - accuracy: 1.0000
Epoch 00003: saving model to cnn_practice
INFO:tensorflow:Assets written to: cnn_practice/assets
1/1 [==============================] - 14s 14s/step - loss: 0.3133 - accuracy: 1.0000 - val_loss: 0.3133 - val_accuracy: 1.0000
Epoch 4/10
1/1 [==============================] -

# Custom CNN Model

In this step, write and train your own convolutional neural network using Keras. You can use any architecture that suits you as long as it has at least one convolutional and one pooling layer at the beginning of the network - you can add more if you want. 

In [0]:
# create the custom cnn model

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Sequential

cnn_model = Sequential()
cnn_model.add(Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)))
cnn_model.add(MaxPooling2D(pool_size=(2,2)))
cnn_model.add(Conv2D(64, (3,3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2,2)))
cnn_model.add(Conv2D(64, (3,3), activation='relu'))
cnn_model.add(Flatten())
cnn_model.add(Dense(64, activation='relu'))
cnn_model.add(Dense(1, activation='sigmoid'))

In [0]:
# Compile the model - from lecture

base_learning_rate = 0.001
model.compile(optimizer = tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [39]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [40]:
# use what we used previously :) for fitting

epochs = 10

batch_size=10

stop = EarlyStopping(monitor='val_loss', mode='min', min_delta=0.001, patience=3, verbose=1)
filepath = 'cnn_practice'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, mode='auto', save_freq='epoch')

history = model.fit(
          X_train, y_train,
          epochs=epochs,
          validation_data=(X_val, y_val),
          verbose=1,
          callbacks=[stop, checkpoint],
          batch_size=batch_size)

Epoch 1/10
1/1 [==============================] - ETA: 0s - loss: 0.3133 - accuracy: 1.0000
Epoch 00001: saving model to cnn_practice
INFO:tensorflow:Assets written to: cnn_practice/assets
1/1 [==============================] - 15s 15s/step - loss: 0.3133 - accuracy: 1.0000 - val_loss: 0.3133 - val_accuracy: 1.0000
Epoch 2/10
1/1 [==============================] - ETA: 0s - loss: 0.3133 - accuracy: 1.0000
Epoch 00002: saving model to cnn_practice
INFO:tensorflow:Assets written to: cnn_practice/assets
1/1 [==============================] - 14s 14s/step - loss: 0.3133 - accuracy: 1.0000 - val_loss: 0.3133 - val_accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - ETA: 0s - loss: 0.3133 - accuracy: 1.0000
Epoch 00003: saving model to cnn_practice
INFO:tensorflow:Assets written to: cnn_practice/assets
1/1 [==============================] - 14s 14s/step - loss: 0.3133 - accuracy: 1.0000 - val_loss: 0.3133 - val_accuracy: 1.0000
Epoch 4/10
1/1 [==============================] -

# Custom CNN Model with Image Manipulations
## *This a stretch goal, and it's relatively difficult*

To simulate an increase in a sample of image, you can apply image manipulation techniques: cropping, rotation, stretching, etc. Luckily Keras has some handy functions for us to apply these techniques to our mountain and forest example. Check out these resources to help you get started: 

1. [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class)
2. [Building a powerful image classifier with very little data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
 

In [0]:
# State Code for Image Manipulation Here

# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language